Bloco 1: Instalação de Dependências

In [1]:
import sys
!{sys.executable} -m pip install pyarrow

Bloco 2: Importação de Bibliotecas

In [2]:
import pandas as pd
import numpy as np
import json

Bloco 3: Carregamento dos Dados Processados

In [3]:
caminho_dados_limpos = '../data/processed/dados_limpos.parquet'
df = pd.read_parquet(caminho_dados_limpos)

print("Dados limpos carregados com sucesso!")
display(df.head())

Dados limpos carregados com sucesso!


,nk_ota_localizer_id,fk_contact,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,gmv_success,total_tickets_quantity_success,datetime_purchase
0,bc02d5245bec63b30ff1102fa273fc03f58bc9cc3f674e...,a7218ff4ee7d37d48d2b4391b955627cb089870b934912...,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,50e9a8665b62c8d68bccc77c7c92431a1aa26ccbd38ed4...,0,0,8527a891e224136950ff32ca212b45bc93f69fbb801c3b...,1,89.09,1,2018-12-26 15:33:35
1,fb3caed9b2f1b6016d45ccddb19095476e61a2c85faa8e...,3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...,7688b6ef52555962d008fff894223582c484517cea7da4...,8c1f1046219ddd216a023f792356ddf127fce372a72ec9...,0,0,ec2e990b934dde55cb87300629cedfc21b15cd28bbcf77...,1,121.99,1,2018-12-21 18:41:54
2,2ee9d0978acb5e113d0b3f846ab3f88c5a426321da8f87...,e15109b0b8e9f6f1554e560837eb55543f035f91d8be4f...,2fca346db656187102ce806ac732e06a62df0dbb2829e5...,be47addbcb8f60566a3d7fd5a36f8195798e2848b36819...,0,0,1d0ebea552eb43d0b1e1561f6de8ae92e3de7f1abec523...,1,188.99,1,2021-02-19 19:11:40
3,92c004fb7013d0b6cc363bf860b63ebcb4ab8827f4c2b2...,ec66b05934a8d00f661257ea91874241fccb4fb128028d...,7688b6ef52555962d008fff894223582c484517cea7da4...,a3af7b3808c4cf72478d05c9bab9c0d47e31c1d2cb3a29...,0,0,3068430da9e4b7a674184035643d9e19af3dc7483e31cc...,1,232.96,1,2020-12-03 09:16:29
4,7fdcd3e03f08ecea34edbe57ac88e0c8c2ef2645eceb40...,e15109b0b8e9f6f1554e560837eb55543f035f91d8be4f...,2fca346db656187102ce806ac732e06a62df0dbb2829e5...,be47addbcb8f60566a3d7fd5a36f8195798e2848b36819...,0,0,1d0ebea552eb43d0b1e1561f6de8ae92e3de7f1abec523...,1,187.57,1,2021-04-09 09:25:08


Bloco 4: Carregar e Aplicar Aliases (dos Mapas JSON)

In [4]:
# Carregar os dicionários de mapeamento
with open('../data/processed/mapa_clientes_alias.json', 'r') as f:
    mapa_clientes = json.load(f)
with open('../data/processed/mapa_locais_alias.json', 'r') as f:
    mapa_locais = json.load(f)

print("Mapeamentos de alias carregados.")

# Aplicar os aliases para criar novas colunas legíveis
df['nome_cliente'] = df['fk_contact'].map(mapa_clientes)
df['origem_alias'] = df['place_origin_departure'].map(mapa_locais)
df['destino_alias'] = df['place_destination_departure'].map(mapa_locais)

# Criar a coluna 'produto' (o trecho legível)
df['produto'] = df['origem_alias'] + ' -> ' + df['destino_alias']

print("Colunas de alias criadas com sucesso!")
display(df[['nome_cliente', 'produto']].head())

Mapeamentos de alias carregados.
Colunas de alias criadas com sucesso!


,nome_cliente,produto
0,Lunna Gomes,Pinto -> Caldeira
1,Henrique Cassiano,Pinto -> Mendonça
2,Otávio Fernandes,Castro -> Gomes da Mata
3,Manuela Freitas,Pinto -> Farias
4,Otávio Fernandes,Castro -> Gomes da Mata


Bloco 5: Engenharia de Features de Comportamento

In [5]:
# Feature 1: tipo_viagem (Ida e Volta vs. Só Ida)
df['tipo_viagem'] = 'Ida e Volta'
df.loc[df['place_origin_return'] == '0', 'tipo_viagem'] = 'Só Ida'

# Feature 2: viaja_sozinho
df['viaja_sozinho'] = np.where(df['total_tickets_quantity_success'] == 1, 'Sim', 'Não')

# Feature 3: compra_fds (Fim de Semana)
df['compra_fds'] = np.where(df['datetime_purchase'].dt.dayofweek >= 5, 'Sim', 'Não')

# Feature 4: compra_alta_temporada
meses_alta_temporada = [12, 1, 7] # Exemplo: Dezembro, Janeiro e Julho
df['compra_alta_temporada'] = np.where(df['datetime_purchase'].dt.month.isin(meses_alta_temporada), 'Sim', 'Não')

print("Novas features de comportamento criadas. Amostra:")
display(df[['fk_contact', 'tipo_viagem', 'viaja_sozinho', 'compra_fds', 'compra_alta_temporada']].head())

Novas features de comportamento criadas. Amostra:


,fk_contact,tipo_viagem,viaja_sozinho,compra_fds,compra_alta_temporada
0,a7218ff4ee7d37d48d2b4391b955627cb089870b934912...,Só Ida,Sim,Não,Sim
1,3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...,Só Ida,Sim,Não,Sim
2,e15109b0b8e9f6f1554e560837eb55543f035f91d8be4f...,Só Ida,Sim,Não,Não
3,ec66b05934a8d00f661257ea91874241fccb4fb128028d...,Só Ida,Sim,Não,Sim
4,e15109b0b8e9f6f1554e560837eb55543f035f91d8be4f...,Só Ida,Sim,Não,Não


Bloco 6: Análise RFM (Agregação por Cliente)

In [6]:
# Usamos o dia seguinte à última compra registrada para garantir que todas as recências sejam positivas.
snapshot_date = df['datetime_purchase'].max() + pd.Timedelta(days=1)

# Agrupando por cliente (fk_contact) e calculando RFM
df_rfm = df.groupby('fk_contact').agg(
    Recencia=('datetime_purchase', lambda date: (snapshot_date - date.max()).days), # Recência
    Frequencia=('nk_ota_localizer_id', 'nunique'), # Frequência (compras únicas)
    ValorMonetario=('gmv_success', 'sum') # Valor Monetário
)

# Renomeando as colunas para o padrão RFM
df_rfm.rename(columns={'datetime_purchase': 'Recencia',
                       'nk_ota_localizer_id': 'Frequencia',
                       'gmv_success': 'ValorMonetario'}, inplace=True)

print("DataFrame RFM (agregado por cliente) criado com sucesso:")
display(df_rfm.head())

DataFrame RFM (agregado por cliente) criado com sucesso:


,Recencia,Frequencia,ValorMonetario
fk_contact,,,
00007a5d618cd250d7f05766cfe01a8663a3767f1cd669eebe57f83f03b51e3e,485,4,688.48
00026c6e9ff2f6387766bc6b4b4f1d1a282c60c9976064ea4e0cf8d5163aca98,249,1,79.44
00026fad47197aa4c6ee22af325a80bf1001209f21021d2755f5cbaf4c453135,373,1,271.79
0002b73c84f0f22a291226e3aec6596de90488f776ef23e9ba15338caeb935a7,1842,1,35.42
0003a32718f6bcdb2bf379f645c23e13f56b7b1d43d9531482e987849df7410f,515,1,95.72


Bloco 7: Persistência dos Resultados (Saídas Finais)

In [7]:
# --- Saída A: O log de VENDAS (transacional) completo, agora limpo e com aliases.
# (Usado pelo app.py e pelo Notebook 06_Recommendation_Model)
print("Alinhando e salvando o arquivo de vendas para o frontend...")
caminho_sales = '../data/redis/sales.parquet'

# Renomeando colunas-chave para o padrão do app
df_para_app = df.rename(columns={
    'fk_contact': 'id_cliente',
    'datetime_purchase': 'data_venda',
    'gmv_success': 'valor_venda'
})

df_para_app.to_parquet(caminho_sales, index=False)
print(f"--> Arquivo 'sales.parquet' salvo em: '{caminho_sales}'")


# --- Saída B: O arquivo AGREGADO (RFM)
# (Usado pelo Notebook 04_Segmentation_Model)
print("\nSalvando o arquivo com as features RFM...")
caminho_rfm = '../data/curated/rfm_features.parquet'
df_rfm.to_parquet(caminho_rfm)
print(f"--> Arquivo RFM ('rfm_features.parquet') salvo em: '{caminho_rfm}'")

Alinhando e salvando o arquivo de vendas para o frontend...
--> Arquivo 'sales.parquet' salvo em: '../data/redis/sales.parquet'

Salvando o arquivo com as features RFM...
--> Arquivo RFM ('rfm_features.parquet') salvo em: '../data/curated/rfm_features.parquet'
